In [85]:
from tefas import Crawler

from time import gmtime, strftime

import pandas as pd

def get_today():
    return strftime('%Y-%m-%d',gmtime())

def download_data(fund: str = 'MAC', market: str = 'TEFAS', start: str = '2025-01-01'):
    tefas_data = Crawler()
    end_date  = strftime('%Y-%m-%d',gmtime())
    print(f'Downloading **{fund}** data from {market} for the dates {start}---{end_date}')
    data = tefas_data.fetch(start=str(start), end=end_date, name=fund, columns=["code", "date", "price", "stock", "number_of_investors"])
    print('Data was fetched from the API!')
    data = data.sort_values('date')
    data = data.reset_index(drop = True)
    if len(data) == 0:
        print(f'Nothing downloaded, be sure if nothing is wrong!')
    else:
        return data    

def get_data(fund: str = 'MAC', market: str = 'TEFAS'):
    filename = f'{market}_{fund}.pkl'
    try: # check if data exists
        # if exists, load the data
        data = pd.read_pickle(filename)
        number_of_elements = len(data)
        last_date_in_data = data.loc[number_of_elements - 1, 'date']
        today = get_today()
        if last_date_in_data == today:
            print('Data is up-to-date, returning the data for use!')
        else:
            print('Data is being updated...')
            recent_data = download_data(fund = fund, market = market, start = last_date_in_data)
            if len(recent_data) > 1:
                recent_data = recent_data.sort_values('date')
                recent_data = recent_data.reset_index(drop = True)
                data = pd.concat([data, recent_data.loc[1,:]], ignore_index=True)
                
                filename = f'{market}_{fund}.pkl'
                print(f'Data saved in {filename}')
                data.to_pickle(filename)
            else: 
                return data
    except FileNotFoundError:
        print("The file does not exist; downloading the data...")   
        data = download_data(fund = fund, market = market, start = '2025-01-01')
        
        filename = f'{market}_{fund}.pkl'
        data.to_pickle(filename)
        print(f'Data saved in {filename}')
    return data

In [86]:
mydata = get_data(fund = 'MAC', market = 'TEFAS')
mydata

Data is being updated...
Data was fetched from the API!


,code,date,price,stock,number_of_investors
0,MAC,2025-01-02,0.571942,92.87,60357.0
1,MAC,2025-01-03,0.573408,92.36,60468.0
2,MAC,2025-01-06,0.576191,92.19,60435.0
3,MAC,2025-01-07,0.579996,92.42,60542.0
4,MAC,2025-01-08,0.578804,91.87,60405.0
...,...,...,...,...,...
57,MAC,2025-03-24,0.575205,97.72,55113.0
58,MAC,2025-03-25,0.582716,98.38,55015.0
59,MAC,2025-03-26,0.606350,95.77,54826.0
60,MAC,2025-03-27,0.602340,96.06,54567.0


In [87]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.line(mydata,x = 'date', y = ['price'])
fig.update_layout(xaxis_title = 'Days',
                  yaxis_title = 'Percentage',
                  title = 'Change of MAC',
                  template = 'simple_white')
fig.show()